In [551]:
import numpy as np
import pandas as pd

In [552]:
train = pd.read_csv('Restaurant.csv')

print (train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        12 non-null     int64 
 1   Alt       12 non-null     object
 2   Bar       12 non-null     object
 3   Fri       12 non-null     object
 4   Hun       12 non-null     object
 5   Pat       10 non-null     object
 6   Price     12 non-null     object
 7   Rain      12 non-null     object
 8   Res       12 non-null     object
 9   Type      12 non-null     object
 10  Est       12 non-null     object
 11  WillWait  12 non-null     object
dtypes: int64(1), object(11)
memory usage: 1.2+ KB
None


In [553]:
print(train.head(10))

   id  Alt  Bar  Fri  Hun   Pat Price Rain  Res     Type     Est WillWait
0   1  Yes   No   No  Yes  Some   $$$   No  Yes   French    0-10      Yes
1   2  Yes   No   No  Yes  Full     $   No   No     Thai   30-60       No
2   3   No  Yes   No   No  Some     $   No   No   Burger    0-10      Yes
3   4  Yes   No  Yes  Yes  Full     $  Yes   No     Thai   10-30      Yes
4   5  Yes   No  Yes   No  Full   $$$   No  Yes   French     >60       No
5   6   No  Yes   No  Yes  Some    $$  Yes  Yes  Italian    0-10      Yes
6   7   No  Yes   No   No   NaN     $  Yes   No   Burger    0-10       No
7   8   No   No   No  Yes  Some    $$  Yes  Yes     Thai    0-10      Yes
8   9   No  Yes  Yes   No  Full     $  Yes   No   Burger     >60       No
9  10  Yes  Yes  Yes  Yes  Full   $$$   No  Yes  Italian   10-30       No


In [554]:
nptest = np.array(train["WillWait"])
nptest.sort()
nptest

array(['No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes'], dtype=object)

In [555]:
YesNo_Mapping = {
    "Yes": 1,
    "No": 0,
}

train["Alt"] = train["Alt"].replace(YesNo_Mapping)
train["Bar"] = train["Bar"].replace(YesNo_Mapping)
train["Fri"] = train["Fri"].replace(YesNo_Mapping)
train["Hun"] = train["Hun"].replace(YesNo_Mapping)
train["Rain"] = train["Rain"].replace(YesNo_Mapping)
train["Res"] = train["Res"].replace(YesNo_Mapping)
train["WillWait"] = train["WillWait"].replace(YesNo_Mapping)

Pat_Mapping = {
    "None": 0,
    "Some": 0.5,
    "Full": 1,
}

train["Pat"] = train["Pat"].replace(Pat_Mapping)

Price_Mapping = {
    "$": 1,
    "$$": 2,
    "$$$": 3,
}
train["Price"] = train["Price"].replace(Price_Mapping)

Type_Mapping = {
    "French": 1,
    "Thai": 2,
    "Burger": 3,
    "Italian": 4,
}
train["Type"] = train["Type"].replace(Type_Mapping)

In [556]:
# print(train[["Type", "WillWait"]].groupby(["Type"], as_index=False).mean())
GroupedByType = train[["Type", "WillWait"]].groupby(["Type"], as_index=False).mean()

print(GroupedByType)

   Type  WillWait
0     1       0.5
1     2       0.5
2     3       0.5
3     4       0.5


In [557]:
GroupedByPat = train[["Pat", "WillWait"]].groupby(["Pat"], as_index=False).mean()

print(GroupedByPat)

   Pat  WillWait
0  0.5  1.000000
1  1.0  0.333333


In [558]:
print (train)

    id  Alt  Bar  Fri  Hun  Pat  Price  Rain  Res  Type     Est  WillWait
0    1    1    0    0    1  0.5      3     0    1     1    0-10         1
1    2    1    0    0    1  1.0      1     0    0     2   30-60         0
2    3    0    1    0    0  0.5      1     0    0     3    0-10         1
3    4    1    0    1    1  1.0      1     1    0     2   10-30         1
4    5    1    0    1    0  1.0      3     0    1     1     >60         0
5    6    0    1    0    1  0.5      2     1    1     4    0-10         1
6    7    0    1    0    0  NaN      1     1    0     3    0-10         0
7    8    0    0    0    1  0.5      2     1    1     2    0-10         1
8    9    0    1    1    0  1.0      1     1    0     3     >60         0
9   10    1    1    1    1  1.0      3     0    1     4   10-30         0
10  11    0    0    0    0  NaN      1     0    0     2    0-10         0
11  12    1    1    1    1  1.0      1     0    0     3   30-60         1


In [559]:
def calculate_entropy(labels):
    label_counts = np.bincount(labels)
    label_probs = label_counts / len(labels)
    entropy = -np.sum(label_probs * np.log2(label_probs + 1e-10))  # Add small value to avoid log(0)
    return entropy

In [560]:
def calculate_gini_index(labels):
    label_counts = np.bincount(labels)
    label_probs = label_counts / len(labels)
    gini_index = 1 - np.sum(label_probs ** 2)
    return gini_index

In [561]:
# The most common value in the target attribute of the given examples
# examples: A numpy arr containing the examples

# from collections import Counter

def PLURALITY_VALUE(examples):
    # classes = examples[:, -1]
    # counts = Counter(classes)
    # return counts.most_common(1)[0][0]
    return np.argmax(np.bincount(examples[:, -1]))

In [562]:
def find_attribute_index (attribute, examples):
    column_names = examples[0]
    index = np.where(column_names == attribute)[0]
    return index

In [563]:
# Selects the best attribute to use for splitting the data based on entropy
# attributes: A set containing the attributes to choose from
# examples: A NumPy array containing the examples

def select_best_attribute_entropy (attributes, examples):
    best_attribute = None
    # best_entropy = float('inf')
    min_entropy = np.inf
    
    for attribute in attributes:
        # examples = np.array(examples)
        # attribute_values = np.unique(examples[:, attribute])
        # attribute_entropy = 0
        
        # for value in attribute_values:
        #     value_examples = examples[examples[:, attribute] == value]
        #     value_prob = len(value_examples) / len(examples)
        #     value_entropy = calculate_entropy(value_examples[:, -1])
            
        #     attribute_entropy += value_prob * value_entropy
        
        # if attribute_entropy < best_entropy:
        #     best_entropy = attribute_entropy
        #     best_attribute = attribute
        entropy = entropy_help(examples[:, find_attribute_index(attribute, examples)])
        if entropy < min_entropy:
            best_attribute = attribute
            min_entropy = entropy
    
    return best_attribute


# Calculates the entropy of the given examples.
# examples: A NumPy array containing the examples
def entropy_help(examples):
    unique_values, counts = np.unique(examples, return_counts=True)
    probabilities = counts / len(examples)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

In [564]:
def select_best_attribute_GiniIndex (attributes, examples):
    best_attribute = None
    best_gini_index = float('inf')
    
    for attribute in attributes:
        # examples = np.array(examples)
        attribute_values = np.unique(examples[:, find_attribute_index(attribute, examples)])
        attribute_gini_index = 0
        
        for value in attribute_values:
            value_examples = examples[examples[:, attribute] == value]
            value_prob = len(value_examples) / len(examples)
            value_gini_index = calculate_gini_index(value_examples[:, -1])
            
            attribute_gini_index += value_prob * value_gini_index
        
        if attribute_gini_index < best_gini_index:
            best_gini_index = attribute_gini_index
            best_attribute = attribute
    
    return best_attribute

In [565]:
# Learns a decision tree from the given examples and attributes
# examples: A numpy arr containing the dataset
# attributes: A set containing the attributes
# parent_examples: A numpy arr containing the parent examples
# returns: A decision tree as a py dictionary

def LEARN_DECISION_TREE (examples, attributes, parent_examples):
    if len(examples) == 0:
        return PLURALITY_VALUE(parent_examples)
    
    elif np.all(examples[:, -1] == examples[0, -1]):
        return examples[0, -1]
    
    elif len(attributes) == 0:
        return PLURALITY_VALUE(examples)
    
    else:
        # A = select_best_attribute_entropy(attributes, examples)
        A = select_best_attribute_GiniIndex(attributes, examples)
        # examples_slice = np.array(examples[:, A])
        A_index = find_attribute_index(A, examples)
        attribute_values = np.unique(examples[:,A_index])
        tree = {A: {}}
        for value in np.unique(examples[:, A_index]):
            exs = examples[examples[:, A] == value]
            subtree = LEARN_DECISION_TREE(exs, attributes - {A}, examples)
            tree[A][value] = subtree
        return tree

In [566]:
# def PRINT_DECISION_TREE(tree, indent=''):
#     attribute = next(iter(tree))
#     print(indent + attribute + ':')

#     subtree = tree[attribute]
#     for value in subtree:
#         if isinstance(subtree[value], dict):
#             print(indent + ' ' + value + ' =>')
#             PRINT_DECISION_TREE(subtree[value, indent + '   '])
#         else:
#             print (indent + '   ' + value + ' => ' + subtree[value])

import json

def print_decision_tree(tree):
  """
  Prints the decision tree to the console.

  Args:
    tree: A decision tree represented as a Python dictionary.
  """

  print(json.dumps(tree, indent=4))



import pydot

def visualize_decision_tree(tree, filename='decision_tree.png'):
  """
  Visualizes the decision tree using PyDot.

  Args:
    tree: A decision tree represented as a Python dictionary.
    filename: The filename of the output PNG image.
  """

  def create_subgraph(node):
    node_id = pydot.Node(json.dumps(node))
    subgraph = dot.subgraph(node_id)

    for child, value in node.items():
      if isinstance(value, dict):
        create_subgraph(child, subgraph)
      else:
        edge = pydot.Edge(node_id, value)
        subgraph.add_edge(edge)

    return subgraph

  dot = pydot.Dot()

  root_subgraph = create_subgraph(tree)
  dot.add_subgraph(root_subgraph)

  dot.write_png(filename)

In [567]:
train_data = np.array(train.head(10))
attributes = {'Alt', 'Bar', 'Fri', 'Hun', 'Pat', 'Price', 'Rain', 'Res', 'Type', 'Est'}
DT = LEARN_DECISION_TREE(train_data, attributes, np.array([]))
# PRINT_DECISION_TREE(DT)
print_decision_tree(DT)
visualize_decision_tree(DT)

{
    "Est": {}
}


AttributeError: 'Dot' object has no attribute 'subgraph'

In [ ]:
def Test_Decision_Tree (tree, test_data):
    predictions = []
    for example in test_data:
        prediction = PREDICT(tree, example, test_data)
        predictions.append(prediction)
    return predictions

def PREDICT(tree, example, examples):
    attribute = next(iter(tree))
    print ("Checking attribute: " + attribute)
    attribute_index = find_attribute_index(attribute, examples)
    value = tuple(example[attribute_index])
    print ("Values is: ")
    print (value)
    print ("tree[attribute] is: " )
    print (tree[attribute])
    if value in tree[attribute]:
        subtree = tree[attribute][value]
        if isinstance(subtree, dict):
            prediction = PREDICT(subtree, example, examples)
        else:
            prediction = subtree
    else:
        # Missing Values ?
        prediction = 20
    return prediction

In [ ]:
test_data = np.array(train.tail(2))[:, :-1]
print (test_data)
predictions = Test_Decision_Tree(DT, test_data)
print(predictions)